<a href="https://colab.research.google.com/github/eb-art/DW_road_sign/blob/master/Matrix3_Day4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
cd '/content/drive/My Drive/Colab Notebooks/matrix/matrix_3/DW_road_sign'

In [0]:
import pandas as pd
import numpy as np

import os
import datetime
import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

#import tensorflow as tf
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
#from tensorflow.keras.utils import to_categorical

# try:
#   # %tensorflow_version only exists in Colab.
#   %tensorflow_version 2.x
# except Exception:
#   pass

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_train.shape, y_train.shape

X_test, y_test = test['features'], test['labels']
X_test.shape, y_test.shape

In [0]:
df = pd.read_csv('data/dw_signnames.csv')
labels_dict = df.to_dict()['b']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def get_cnn_v1(input_shape, num_classes):
  return Sequential([
  Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Flatten(),
  Dense(num_classes, activation='softmax')                    
])
  
def get_cnn_v2(input_shape, num_classes):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  MaxPool2D(),
  Dropout(0.5),
  
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.5),

  Flatten(),

  Dense(1024, activation='relu'),
  Dropout(0.5),

  Dense(num_classes, activation='softmax')                    
])
  
def train_model_param(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', 
              optimizer = 'Adam',
              metrics=['accuracy'])
  
  log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

  #logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
  #tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir,histogram_freq=1)
   
  model.fit(
      X_train, y_train,
      batch_size = params_fit.get('batch_size', 128),
      epochs = params_fit.get('epochs', 5),
      verbose = params_fit.get('verbose', 1),
      validation_data = params_fit.get('validation_data', (X_train, y_train)),
      callbacks = [tensorboard_callback]
  )
  return model


In [0]:
# get_cnn_v2(input_shape, num_classes).summary()  
#params_fit = {'epochs': 25}
model = get_cnn_v1(input_shape, num_classes)
model_trained = train_model_param(model, X_train, y_train)
#logdir
%tensorboard --logdir logs/fit

In [0]:
plt.imshow(X_test[400])

In [0]:
y_pred_prob = model_trained.predict(X_test)
y_pred_prob[400]

In [0]:
labels_dict[ np.argmax(y_pred_prob[400])]

In [0]:
#Suma powinna być bliska 1, ale jedna klasa ma przewagę
np.sum([1.5767952e-04, 9.9849033e-01, 4.9064082e-14, 1.2105372e-12,
       1.3517361e-03, 1.6508359e-07, 6.1247123e-23, 2.0586473e-12,
       7.2217652e-24, 2.8086266e-15, 3.5524616e-19, 8.6534007e-20,
       1.3037522e-24, 1.1216663e-21, 4.1743947e-24, 1.4077906e-23,
       2.0241870e-14, 6.6981340e-29, 8.4766314e-16, 1.8029775e-37,
       9.8711307e-17, 1.1187249e-16, 2.3190204e-29, 1.6348336e-33,
       1.3865917e-34, 1.8569403e-31, 3.3224795e-20, 0.0000000e+00,
       3.3444607e-23, 1.6897733e-20, 0.0000000e+00, 1.4937674e-24,
       6.8068947e-34, 0.0000000e+00, 0.0000000e+00, 2.7614104e-33,
       0.0000000e+00, 2.0642238e-37, 4.5392048e-31, 0.0000000e+00,
       1.6168093e-35, 1.1680189e-27, 4.6438755e-22])

In [0]:
plt.bar(range(43), y_pred_prob[400], log=True)
# w liniowej mamy jeden bin dla tej klasy i niewidoczne pozostałe

In [0]:
# model_trained - aby nie myliło się z funkcja trenującą model
def predict(model_trained, X_test, y_test, scoring = accuracy_score):
  
  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis = 1) #szuka wiersz po wierszu

  y_test_norm = np.argmax(y_test, axis = 1)

  return scoring(y_test_norm, y_pred) #accuracy_score(y_test_norm, y_pred)

In [0]:
predict(model_trained, X_test, y_test)

In [0]:
# architecture engineering

def get_cnn_v3(input_shape, num_classes):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  MaxPool2D(),
  Dropout(0.3),
  
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Flatten(),

  Dense(1024, activation='relu'),
  Dropout(0.3),

  Dense(num_classes, activation='softmax')                    
])
  
#model = get_cnn_v1(input_shape, num_classes)

def train_and_predict(model):
  model_trained = train_model_param(model, X_train, y_train) 

  return predict(model_trained, X_test, y_test) 

train_and_predict(get_cnn_v3(input_shape, num_classes))

In [0]:
# architecture engineering

def get_cnn_v4(input_shape, num_classes):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),
  
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Flatten(),

  Dense(1024, activation='relu'),
  Dropout(0.3),

  Dense(num_classes, activation='softmax')                    
])

train_and_predict(get_cnn_v4(input_shape, num_classes))

In [0]:
# problem
# flatten miał 5x5 - można doprowadzić do za dużego zmniejszenia
# dlatego dodaje się funkcje padding
# np. dodanie zer, lub duplikacja pikseli zeby nie spadł do 0

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

def get_cnn_v5(input_shape, num_classes):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
  MaxPool2D(),
  Dropout(0.3),
  
  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Flatten(),

  Dense(1024, activation='relu'),
  Dropout(0.3),

  Dense(num_classes, activation='softmax')                    
])

train_and_predict(get_cnn_v5(input_shape, num_classes))

In [0]:
# dodajemy blok dense - trochę gorze, może za duza siec? 
def get_cnn_v6(input_shape, num_classes):
  return Sequential([
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
  MaxPool2D(),
  Dropout(0.3),
  
  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Flatten(),

  Dense(1024, activation='relu'),
  Dropout(0.3),

  Dense(1024, activation='relu'),
  Dropout(0.3),

  Dense(num_classes, activation='softmax')                    
])

train_and_predict(get_cnn_v6(input_shape, num_classes))

In [0]:
# dodajemy blok dense - trochę gorze, może za duza siec? 
def get_cnn_v6(input_shape, num_classes):
  return Sequential([
  Conv2D(filters=32, kernel_size=(5,5), activation='relu', input_shape=input_shape),
  Conv2D(filters=32, kernel_size=(5,5), activation='relu', padding='same'),
  MaxPool2D(),
  Dropout(0.3),
  
  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
  Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
  MaxPool2D(),
  Dropout(0.3),

  Flatten(),

  Dense(1024, activation='relu'),
  Dropout(0.3),

  Dense(512, activation='relu'),
  Dropout(0.3),

  Dense(num_classes, activation='softmax')                    
])

train_and_predict(get_cnn_v6(input_shape, num_classes))

In [0]:
# obrazy mają 3 kanały skali kolorów: RGB
X_train[0].shape
# konwertujemy do skali szarości
print(color.rgb2gray(X_train[0]).shape)
plt.imshow(color.rgb2gray(X_train[0]));
# trzeba dodać skalę kolorystyczną, bo obraz ma shape 32 x 32 x 1

In [0]:
plt.imshow(color.rgb2gray(X_train[0]), cmap = plt.get_cmap('gray'));

In [0]:
# można wszystkie zrobić
color.rgb2gray(X_train).shape

In [0]:
color.rgb2gray(X_train).reshape(-1, 32, 32, 1).shape

In [0]:
X_train_gray = color.rgb2gray(X_train)
X_test_gray = color.rgb2gray(X_test)

In [0]:
# trzeba zrobić reshape, żeby pasował do struktur sieci
# -1 to ostatnia wartość
X_train_gray = color.rgb2gray(X_train).reshape(-1, 32, 32, 1)
X_test_gray = color.rgb2gray(X_test).reshape(-1, 32, 32, 1)

In [0]:
input_shape2 = (32, 32, 1)

model = get_cnn_v6(input_shape2, num_classes)
model_trained = train_model_param(model, X_train_gray, y_train, params_fit={})
predict(model_trained, X_test_gray, y_test)

In [0]:
def preproc_img(img):
  hsv = color.rgb2hsv(img)
  hsv[:,:,2] = exposure.equalize_adapthist(hsv[:,:,2])
  img = color.hsv2rgb(hsv)

  return img

In [0]:
plt.imshow(X_train[400])

In [0]:
plt.imshow(preproc_img(X_train[400]))

In [0]:
https://github.com/joshwadd/Deep-traffic-sign-classification